Here I try saving the targets as a sparse array (so they can be reasonably stored in the browser)

In the end sparse array were larger so I tried a normal array but this crashes the browser. I found that a binary file of Int16 works OK, as long as I load it as Int16.

In [1]:
from tqdm import tqdm
from path import Path
import json
import time
from pprint import pprint

import numpy as np
import pandas as pd
import re
import logging

basedir=r'/media/isisilon/Data/My_Documents/Documents/eclipse-workspace/bb2/mhwebui2/mhwebui2/scripts/'
mhpath = Path(os.path.abspath("../vendor/makehuman-commandline/makehuman"))


In [2]:
# from .mh_helpers import clean, short_hash, clean_modifier




#===============================================================================
# Import Makehuman resources, needs to be with makehuman dir as current dir
#===============================================================================

appcwd = os.path.abspath(os.curdir)
sys.path.append(mhpath)
sys.path.append(appcwd)
sys.path.append('.')

def getHuman():
    """Load a human model with modifiers."""
    with mhpath:
        # maxFaces *uint* Number of faces per vertex (pole), None for default (min 4)
        human = Human(files3d.loadMesh(
            getpath.getSysDataPath("3dobjs/base.obj"),
            maxFaces=5))
        # load modifiers onto human
        humanmodifier.mods_loaded = False
        modifiers = humanmodifier.loadModifiers(
            getpath.getSysDataPath('modifiers/modeling_modifiers.json'), human)
        return human

with mhpath:
    import makehuman
    oldpath = os.sys.path
    makehuman.set_sys_path()
    # make makehuman paths absolute by going through newest paths and making abs
    for i in range(len(os.sys.path)):
        p = os.sys.path[i]
        if p[0:2] == './':
            os.sys.path[i] = os.path.join(
                os.path.abspath('.'), p.replace('./', ''))
        else:
            break

    makehuman.init_logging()
    logging.getLogger().setLevel(logging.CRITICAL)
    
    import proxy as mhproxy
    import humanargparser
    import targets as mhtargets
    from human import Human
    import files3d
    import algos3d
    import getpath
    import humanmodifier
    from core import G
    import headless
    import autoskinblender
    import export
    
    # Init console app
    with mhpath:
        G.app = headless.ConsoleApp()
    G.app.selectedHuman = human = getHuman()
    headless.OBJExporter = None
    headless.MHXExporter = None
    headless.MhxConfig = None
    humanargparser.mods_loaded = False


Initialized logging


In [4]:
targets = mhtargets.getTargets()
target_files = sorted([t.path for t in targets.targets])
len(target_files)

1258

In [5]:
human = getHuman()

In [6]:
# load sparse matrix
import scipy as sp
from scipy.sparse import csr_matrix

In [41]:
npzname = mhtargets.getSysDataPath('targets.npz') 
target_data = np.load(npzname)


# load targets into sparse matrix
row = []
col = []
data = []
for i, target_path in enumerate(target_files):
    index_key = target_path.replace('.target','.index').replace('data/','')
    vector_key = target_path.replace('.target','.vector').replace('data/','')
    inds = target_data[index_key]
    d=target_data[vector_key]
    
    for ind,dd in zip(inds,d):
        for v,ddd in enumerate(dd):
            if ddd!=0:
                col+=[i]
                row+=[ind*3+v]
                data+=[int(ddd)]

In [100]:
n=len(human.mesh.coord)*3
m=len(target_files)
target_sparse = csr_matrix((data, (col,row)), shape=(m,n), dtype=np.int16).toarray()
target_sparse.shape

In [20]:
# test
def calc_new_verts(target_weights):
    """calculate the change in vertices for a specific collection of targets"""
    return np.dot(target_weights,target_sparse).reshape((-1,3)).astype(np.float32)*1e-3

target_weights = np.random.random(target_sparse.shape[0])-0.5
d_vert = calc_new_verts(target_weights)

vertices_updates = (target_sparse.sum(0)>0).sum()# how many vertices updated
assert vertices_updates>10000, 'shouldupdate more thank 10k vertices'
assert (d_vert.sum(0)!=0).sum()==3, 'should update one of each vertice'

In [21]:
# test
n=80
target_weights = np.zeros(target_sparse.shape[0])
target_weights[n]=1
d_vert = calc_new_verts(target_weights)

# load a target manually and compare
target_path = target_files[n]
index_key = target_path.replace('.target','.index').replace('data/','')
vector_key = target_path.replace('.target','.vector').replace('data/','')
inds = target_data[index_key]
d=target_data[vector_key]
coords = np.zeros_like(human.mesh.coord)
coords[inds]=d*1e-3

np.testing.assert_array_almost_equal(coords, d_vert)

In [76]:
with open('/tmp/targets_data.bin','wb') as fo:
    fo.write(target_sparse.tobytes())